In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/')

In [ ]:
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
def perform_hough_transform(variable, threshold, opening_kernel_pixels):

    # Handle variable
    if isinstance(variable, str):
        src = cv.imread(variable, cv.IMREAD_GRAYSCALE)
    elif isinstance(variable, np.ndarray):
        src = variable
    else:
        raise ValueError("Variable is neither a string nor a np.array.")

    # Perform Opening
    if opening_kernel_pixels is not None:
        # Perform erosion
        kernel = np.ones((opening_kernel_pixels,opening_kernel_pixels),np.uint8)
        erosion = cv.erode(src,kernel,iterations = 1)
        # Perform dilation
        src = cv.dilate(erosion,kernel,iterations = 1) # reassign opened image to src-variable


    dst = cv.Canny(src, 200, 240, None, 3)

    cdstP = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)

    # save angles of all lines in angles
    angles = []

    # threshold from 50 to 500
    linesP = cv.HoughLinesP(dst, 1, np.pi / 180, threshold, None, 50, 10)

    if linesP is not None:
        for i in range(0, len(linesP)):
            l = linesP[i][0]

            angle = np.arctan2(l[3] - l[1], l[2] - l[0]) * 180.0 / np.pi
            angles.append(angle)

            cv.line(cdstP, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv.LINE_AA)

    dominant_angle = None
    if angles:
        angles = np.array(angles)
        hist, bin_edges = np.histogram(angles, bins=180)  # Create a histogram of angles
        dominant_bin = np.argmax(hist)  # Find the bin with the highest frequency
        dominant_angle = bin_edges[dominant_bin]  # Get the dominant angle

    return dominant_angle, cdstP

def write_angle_in_original_image(variable, dominant_angle):

    # Handle variable
    if isinstance(variable, str):
        src = cv.imread(variable, cv.IMREAD_GRAYSCALE)
    elif isinstance(variable, np.ndarray):
        src = variable
    else:
        raise ValueError("Variable is neither a string nor a np.array.")

    #annotated_image = cv.imread(path, cv.IMREAD_GRAYSCALE)
    cv.putText(src, f"Predicted Angle: {dominant_angle:.2f} degrees", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)

    return src

def draw_angle_in_original_image(variable, dominant_angle):

    # Handle variable
    if isinstance(variable, str):
        src = cv.imread(variable, cv.IMREAD_GRAYSCALE)
    elif isinstance(variable, np.ndarray):
        src = variable
    else:
        raise ValueError("Variable is neither a string nor a np.array.")

    #image_w_lines = cv.imread(path, cv.IMREAD_GRAYSCALE)
    angle_rad = np.radians(dominant_angle)
    rows, cols = src.shape

    cv.line(src, ((90*cols) // 100, (90*rows) // 100),
            (int((90*cols) // 100 + 100 * np.sin(angle_rad)), int((90*rows) // 100 - 100 * np.cos(angle_rad))),
            (255, 255, 255), 2, cv.LINE_AA)
    cv.line(src, ((90*cols) // 100, (90*rows) // 100),
            (int((90*cols) // 100 + 100 * np.cos(angle_rad)), int((90*rows) // 100 + 100 * np.sin(angle_rad))),
            (255, 255, 255), 2, cv.LINE_AA)

    return src

def rotate_image_by_dominant_angle(variable, dominant_angle, crop_image):

    # Handle variable
    if isinstance(variable, str):
        src = cv.imread(variable)
    elif isinstance(variable, np.ndarray):
        src = variable
    else:
        raise ValueError("Variable is neither a string nor a np.array.")

    # Check if the image is grayscale or color
    if len(src.shape) == 2:  # Grayscale image
        channels = [src]
    elif len(src.shape) == 3:  # Color image
        channels = [src[:, :, i] for i in range(src.shape[2])]
    else:
        raise ValueError("Unsupported number of image channels.")

    # Rotate each channel separately
    rotated_channels = []
    for channel in channels:
        rows, cols = channel.shape
        M = cv.getRotationMatrix2D((cols / 2, rows / 2), dominant_angle, 1)
        rotated_channels.append(cv.warpAffine(channel, M, (cols, rows)))
    #rotated_channels = [cv.warpAffine(channel, cv.getRotationMatrix2D((channel.shape[1] / 2, channel.shape[0] / 2), dominant_angle, 1), (channel.shape[1], channel.shape[0])) for channel in channels]

    # Combine the rotated channels
    rotated_img = np.stack(rotated_channels, axis=-1)

    if crop_image:
        # Crop image
        contours, _ = cv.findContours(rotated_img[:, :, 0], cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE) # find contours
        if contours:
            largest_contour = max(contours, key=cv.contourArea)
            x, y, w, h = cv.boundingRect(largest_contour)
            cropped_image = rotated_img[y:y+h, x:x+w, :]
            return cropped_image
        else:
            return rotated_img
    else:
        return rotated_img

In [ ]:
dirname = os.path.dirname(__file__)
folder_path = os.path.join(dirname,'correct_path_to_dataset/3_deskewing/3_deskewing')

for_final_output = []

# Check if the folder path exists
if os.path.exists(folder_path):
    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Check if the file is an image (you can extend this check based on image extensions)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            # Perform operations on the image using OpenCV
            angle, image_w_hough_lines = perform_hough_transform(file_path, 75 , 25)
            annotated_image = write_angle_in_original_image(file_path, angle)
            image_w_lines = draw_angle_in_original_image(file_path, angle)
            rotated_img = rotate_image_by_dominant_angle(file_path,angle, True).astype(np.uint8)

            for_final_output.append({'hough_lines':image_w_hough_lines,'annotated_image':annotated_image,'image_w_lines':image_w_lines,'rotated_img':rotated_img,})

else:
    print("Folder path does not exist.")

In [ ]:
# Plotting the images
part_of_output = for_final_output[9:20] # 10th to 20th elements
num_rows = len(part_of_output)
num_cols = 4

fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 5*num_rows))

for i, row_images in enumerate(part_of_output):
    for j, (image_name, image_array) in enumerate(row_images.items()):
        axes[i, j].imshow(image_array, cmap='gray')  # Assuming grayscale images
        axes[i, j].set_title(image_name)
        axes[i, j].axis('off')

plt.tight_layout()
plt.show()

# Create Checkpoint Output

In [28]:
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt


folder_path = '/content/drive/My Drive/ipda/dataset/3_deskewing/3_deskewing/'
# img_name: fake_id_0_back or fake_id_0__front to fake_id_49_back or fake_id_49__front
img_name = 'fake_id_41__front.png'

# Check if the folder path exists
if os.path.exists(folder_path):
    img_path = os.path.join(folder_path,img_name)
    angle, image_w_hough_lines = perform_hough_transform(img_path, 75 , 25)
    annotated_image = write_angle_in_original_image(img_path, angle)
    image_w_lines = draw_angle_in_original_image(img_path, angle)
    rotated_img = rotate_image_by_dominant_angle(img_path,angle,False).astype(np.uint8)
    img = cv.imread(img_path, cv.IMREAD_GRAYSCALE).astype(np.uint8)
else:
    print("Folder path does not exist.")

In [ ]:
# Display the images using Matplotlib
plt.figure(figsize=(15, 5))

# Original test image
plt.subplot(1, 3, 1)
plt.title("Original Image")
plt.imshow(annotated_image, cmap='gray', extent=[0, img.shape[1], img.shape[0], 0])
plt.axis('off')

# Original test image
plt.subplot(1, 3, 2)
plt.title("Original Image w/ Lines")
plt.imshow(image_w_lines, cmap='gray', extent=[0, img.shape[1], img.shape[0], 0])
plt.axis('off')

# Original test image resized
plt.subplot(1, 3, 3)
plt.title("Original Image Rotated and Cropped")
plt.imshow(rotated_img, cmap='gray', extent=[0, rotated_img.shape[1], rotated_img.shape[0], 0])
plt.axis('off')